In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

df = pd.read_excel('sales_and_eodStocks.xlsx', sheet_name = 'Sheet1') 

result_detect_rmse = df.groupby('Product_ID').agg(
    sales_count=('Product_ID', 'count'),
    total_demand=('Sales', 'sum'),
    first_stock=('EndOfDayStock', 'first'),
    first_sale=('Sales', 'first')
).reset_index()

result_detect_rmse['total_forecast'] = result_detect_rmse['first_stock'] + result_detect_rmse['first_sale']
result_detect_rmse['mean_forecast'] = result_detect_rmse['total_forecast'] / result_detect_rmse['sales_count']
rmse_array = []

for product_id in df['Product_ID'].unique():
    rmse = 0
    product_row = result_detect_rmse.loc[result_detect_rmse['Product_ID'] == product_id]
    product_sales = df[df['Product_ID'] == product_id]['Sales'].tolist()
    for demand in product_sales:
        subt = product_row['mean_forecast'].values[0] - demand
        subt *= subt
        rmse += subt
    rmse = rmse / product_row['sales_count'].values[0]
    rmse = np.sqrt(rmse)
    rmse_array.append(rmse)
    
result_detect_rmse['rmse_value'] = rmse_array
result_detect_rmse['situation'] = pd.cut(result_detect_rmse['rmse_value'], bins=[0, 3, float('inf')],
                                labels=['good amount', 'wrong quantity'], right=False)

result_detect_rmse = result_detect_rmse[result_detect_rmse['situation'] == "good amount"]
result_detect_rmse_sorted = result_detect_rmse.sort_values(by='total_demand', ascending=False)
result_detect_rmse_sorted.to_json('predict_with_rmse.json', orient='records', lines=True)